In [ ]:
%matplotlib widget

import os
import pathlib
import pprint
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict

from Visualizer.DataSet import DataSet
from Visualizer.Monitoring.Persistence.ProfilerSession import ProfilerSession
from Visualizer.Plotting.Plot import Plot

In [ ]:
source_directory = pathlib.Path().parent.absolute()
project_directory = f"{source_directory}/.."
root_directory = f"{project_directory}/.."
resources_directory = f"{root_directory}/Resources"
database = f"{resources_directory}/Database.sqlite"

# Load all profiler sessions
complete_data_set = DataSet(ProfilerSession.load_all(database))

In [ ]:
overview_plot = complete_data_set.table_plot
overview_plot.maximum_rows = 500
overview_plot.plot()

In [ ]:
begin = 142
num_of_policies = 3
num_of_profiles = 1
iterations = 15
energy_dict = OrderedDict({})
timespan_dict = OrderedDict({})

for profiler_session_id in range(begin, begin + num_of_policies*num_of_profiles*iterations,  num_of_profiles):

    data_set = DataSet([profiler_session for profiler_session in complete_data_set.data if profiler_session.id == profiler_session_id])
 
    energy_consumption = next(reversed(data_set.data[0].energy_consumption()['Node'].values()))
    timespan = next(reversed(data_set.data[0].timespan()['Runtime'].values()))
    
    label = data_set.data[0].summary['Label']
    if not label in energy_dict:
        energy_dict[label] = []
        timespan_dict[label] = []
    if energy_consumption > 0:
        energy_dict[label].append(energy_consumption)
    timespan_dict[label].append(timespan)

    
    print(profiler_session_id, label, energy_consumption, timespan)

In [ ]:
print(energy_dict)
avg_energy_list = [np.mean(energy_dict[key]) for key in energy_dict]
print(np.round(avg_energy_list, 1))
energy_percentage_list =[(avg_energy-avg_energy_list[0])/avg_energy_list[0]*100 for avg_energy in avg_energy_list]
print(np.round(energy_percentage_list, 1))
    

In [ ]:
print(timespan_dict)
avg_timespan_list = [np.mean(timespan_dict[key]) for key in timespan_dict]
print(np.round(avg_timespan_list, 1))
timespan_percentage_list =[(avg_timespan-avg_timespan_list[0])/avg_timespan_list[0]*100 for avg_timespan in avg_timespan_list]
print(np.round(timespan_percentage_list, 1))

In [ ]:
def autolabel(rects, xpos='center'):
    """
    Attach a text label above each bar in *rects*, displaying its height.

    *xpos* indicates which side to place the text w.r.t. the center of
    the bar. It can be one of the following {'center', 'right', 'left'}.
    """

    ha = {'center': 'center', 'right': 'left', 'left': 'right'}
    offset = {'center': 0, 'right': 1, 'left': -1}

    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(offset[xpos]*3, 3),  # use 3 points offset
                    textcoords="offset points",  # in both directions
                    ha=ha[xpos], va='bottom')


In [ ]:
no_action = []
minmax = []
system = []
no_action_std = []
minmax_std = []
system_std = []
applicaties = []
avg = 0
for energy in energy_dict:
    label = energy.split(' ')
    if len(label) == 1:
        applicaties.append(label[0])
        avg = np.mean(energy_dict[energy])
        std_perc = np.std([(x-avg)/avg * 100 for x in energy_dict[energy]])
        no_action.append(100)
        no_action_std.append(std_perc)
        continue
 
    percentage = [(x-avg)/avg * 100 + 100 for x in energy_dict[energy]]

    per_avg = np.mean(percentage)
    per_std = np.std(percentage)
    if(label[2] == 'MinMax)'):
        minmax.append(per_avg)
        minmax_std.append(per_std)
    elif(label[2] == 'System)'):
        system.append(per_avg)
        system_std.append(per_std)

ind = np.arange(len(no_action))
width = 0.2

fig, ax = plt.subplots()
rects1 = ax.bar(ind - width, np.round(no_action, 1), width, yerr=no_action_std,
                label='No action')
rects2 = ax.bar(ind, np.round(minmax, 1), width, yerr=minmax_std,
                label='MinMax')
rects3 = ax.bar(ind + width, np.round(system, 1), width, yerr=system_std,
                label='System')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Percentage(%)')
ax.set_title('Energy Consumption')
ax.set_xticks(ind)
names = [x.replace('Profiler', '') for x in applicaties]
ax.set_xticklabels(names)
ax.legend()

autolabel(rects1, "left")
autolabel(rects2, "center")
autolabel(rects3, "right")

fig.tight_layout()

plt.show()